# CCPFS Pipeline Demo

**Capacity-Constrained Personalised Follow-Up Scheduling**

This notebook runs the full scheduling pipeline on synthetic data:
1. Generate a synthetic heart failure discharge cohort
2. Run all scheduling policies (ILP, greedy, baselines)
3. Compare policies on key metrics
4. Sensitivity analysis (capacity, cost ratio, uncertainty)
5. Visualise example patient recommendations

In [ ]:
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", font_scale=1.1)
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["figure.dpi"] = 120

## 1. Generate Synthetic Cohort

500 patients with Weibull-distributed survival curves across three risk groups:
- **High risk** (25%): 30-day event rate ~35%
- **Medium risk** (50%): 30-day event rate ~15%
- **Low risk** (25%): 30-day event rate ~5%

In [ ]:
from evaluation.synthetic import generate_synthetic_cohort

cohort = generate_synthetic_cohort(n_patients=500, seed=42)

survival_curves = cohort["survival_curves"]
survival_stds = cohort["survival_stds"]
event_times = cohort["event_times"]
event_indicators = cohort["event_indicators"]
risk_groups = cohort["risk_groups"]
is_hf = cohort["is_heart_failure"]

n = len(event_times)
event_rate = event_indicators.mean()
print(f"Cohort size: {n} patients")
print(f"Heart failure: {is_hf.sum()} ({is_hf.mean():.0%})")
print(f"30-day event rate: {event_rate:.1%}")
print(f"Risk groups - High: {(risk_groups==2).sum()}, Med: {(risk_groups==1).sum()}, Low: {(risk_groups==0).sum()}")
print(f"Median event time (among events): {np.median(event_times[event_indicators==1]):.0f} days")

### 1.1 Survival Curves by Risk Group

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
days = np.arange(0, 31)
colours = {0: "#2ecc71", 1: "#f39c12", 2: "#e74c3c"}
labels = {0: "Low risk", 1: "Medium risk", 2: "High risk"}

for g in [0, 1, 2]:
    mask = risk_groups == g
    mean_curve = survival_curves[mask].mean(axis=0)
    std_curve = survival_curves[mask].std(axis=0)
    ax.plot(days, mean_curve, color=colours[g], linewidth=2, label=labels[g])
    ax.fill_between(days, mean_curve - std_curve, mean_curve + std_curve,
                    color=colours[g], alpha=0.15)

ax.set_xlabel("Days since discharge")
ax.set_ylabel("Survival probability S(t)")
ax.set_title("Mean Survival Curves by Risk Group")
ax.legend()
ax.set_xlim(0, 30)
ax.set_ylim(0, 1.05)
plt.tight_layout()
plt.show()

## 2. Run All Scheduling Policies

In [ ]:
from evaluation.simulate import run_all_policies, results_summary_table

DAILY_CAPACITY = 10  # 10 slots/day * 30 days = 300 slots for 500 patients
capacity = np.full(30, DAILY_CAPACITY)

results = run_all_policies(
    survival_curves=survival_curves,
    event_times=event_times,
    event_indicators=event_indicators,
    capacity_per_day=capacity,
    survival_stds=survival_stds,
    is_heart_failure=is_hf,
    uncertainty_alpha=0.8,
)

print(results_summary_table(results))

### 2.1 Event-Before-Follow-Up Rate (Primary Metric)

In [ ]:
policy_names = list(results.keys())
ebf_rates = [results[p]["ebf"]["ebf_rate"] for p in policy_names]
catch_rates = [results[p]["ebf"]["catch_rate"] for p in policy_names]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# EBF rate (lower is better)
colours_bar = ["#e74c3c" if "CCPFS" not in p else "#2980b9" for p in policy_names]
bars = ax1.barh(policy_names, ebf_rates, color=colours_bar)
ax1.set_xlabel("Event-Before-Follow-Up Rate (lower is better)")
ax1.set_title("EBF Rate by Policy")
for bar, val in zip(bars, ebf_rates):
    ax1.text(bar.get_width() + 0.002, bar.get_y() + bar.get_height()/2,
             f"{val:.3f}", va="center", fontsize=9)

# Catch rate (higher is better)
bars2 = ax2.barh(policy_names, catch_rates, color=colours_bar)
ax2.set_xlabel("Event Catch Rate (higher is better)")
ax2.set_title("Proportion of Events Caught Before Follow-Up")
for bar, val in zip(bars2, catch_rates):
    ax2.text(bar.get_width() + 0.002, bar.get_y() + bar.get_height()/2,
             f"{val:.3f}", va="center", fontsize=9)

plt.tight_layout()
plt.show()

### 2.2 Schedule Distribution: Where Are Patients Assigned?

In [ ]:
policies_to_compare = ["Uniform-14", "Risk-Bucket", "CCPFS-ILP"]
policies_to_compare = [p for p in policies_to_compare if p in results]

fig, axes = plt.subplots(1, len(policies_to_compare), figsize=(5 * len(policies_to_compare), 4),
                         sharey=True)
if len(policies_to_compare) == 1:
    axes = [axes]

for ax, pname in zip(axes, policies_to_compare):
    hist = results[pname]["distribution"]["day_histogram"]
    ax.bar(range(1, 31), hist, color="#3498db", alpha=0.8)
    ax.axhline(y=DAILY_CAPACITY, color="red", linestyle="--", alpha=0.7, label=f"Capacity ({DAILY_CAPACITY})")
    ax.set_xlabel("Follow-up day")
    ax.set_title(pname)
    ax.legend(fontsize=8)
    mean_d = results[pname]["distribution"]["mean_day"]
    ax.text(0.95, 0.95, f"Mean: {mean_d:.1f}d", transform=ax.transAxes,
            ha="right", va="top", fontsize=9, bbox=dict(boxstyle="round", fc="wheat", alpha=0.5))

axes[0].set_ylabel("Patients")
fig.suptitle("Schedule Distribution by Policy", fontsize=13)
plt.tight_layout()
plt.show()

### 2.3 Expected Cost Comparison

In [ ]:
mean_costs = [results[p]["cost"]["mean_cost"] for p in policy_names]

fig, ax = plt.subplots(figsize=(10, 5))
colours_cost = ["#e74c3c" if "CCPFS" not in p else "#2980b9" for p in policy_names]
bars = ax.barh(policy_names, mean_costs, color=colours_cost)
ax.set_xlabel("Mean Expected Cost per Patient (EUR)")
ax.set_title("Expected Cost by Policy")
for bar, val in zip(bars, mean_costs):
    ax.text(bar.get_width() + 10, bar.get_y() + bar.get_height()/2,
             f"\u20ac{val:,.0f}", va="center", fontsize=9)
plt.tight_layout()
plt.show()

## 3. Sensitivity Analysis

### 3.1 Capacity Sensitivity (H3)

**Hypothesis H3**: Under tighter capacity constraints, the advantage of optimised allocation over uniform scheduling increases.

In [ ]:
from evaluation.sensitivity import vary_capacity

capacities = [5, 10, 15, 20, 30, 50, 999]
cap_results = vary_capacity(
    survival_curves=survival_curves,
    event_times=event_times,
    event_indicators=event_indicators,
    capacities=capacities,
    survival_stds=survival_stds,
    is_heart_failure=is_hf,
)

# Plot EBF rate vs capacity for key policies
fig, ax = plt.subplots(figsize=(10, 6))
plot_policies = ["Uniform-14", "Risk-Bucket", "Guideline", "CCPFS-ILP", "CCPFS-Greedy"]
styles = {
    "Uniform-14": ("--", "o", "#e74c3c"),
    "Risk-Bucket": ("--", "s", "#f39c12"),
    "Guideline": ("--", "^", "#9b59b6"),
    "CCPFS-ILP": ("-", "D", "#2980b9"),
    "CCPFS-Greedy": ("-", "v", "#27ae60"),
}

for pname in plot_policies:
    ebf_vals = []
    valid_caps = []
    for cap in capacities:
        if pname in cap_results[cap]:
            ebf_vals.append(cap_results[cap][pname]["ebf"]["ebf_rate"])
            valid_caps.append(cap)
    if ebf_vals:
        ls, marker, c = styles.get(pname, ("-", "o", "gray"))
        ax.plot(valid_caps, ebf_vals, ls, marker=marker, color=c,
                label=pname, linewidth=2, markersize=7)

ax.set_xlabel("Daily Capacity (slots per day)")
ax.set_ylabel("Event-Before-Follow-Up Rate")
ax.set_title("H3: Scheduling Advantage Grows as Capacity Shrinks")
ax.legend(loc="upper right")
ax.set_xscale("log")
ax.set_xticks(capacities)
ax.set_xticklabels([str(c) if c < 999 else "\u221e" for c in capacities])
plt.tight_layout()
plt.show()

### 3.2 Cost Ratio Sensitivity

In [ ]:
from evaluation.sensitivity import vary_cost_ratio

ratios = [10, 33, 67, 100, 200]
ratio_results = vary_cost_ratio(
    survival_curves=survival_curves,
    event_times=event_times,
    event_indicators=event_indicators,
    ratios=ratios,
    survival_stds=survival_stds,
    is_heart_failure=is_hf,
)

fig, ax = plt.subplots(figsize=(10, 6))
for pname in ["Uniform-14", "Risk-Bucket", "CCPFS-ILP"]:
    costs = []
    valid_ratios = []
    for r in ratios:
        if pname in ratio_results[r]:
            costs.append(ratio_results[r][pname]["cost"]["mean_cost"])
            valid_ratios.append(r)
    if costs:
        ls, marker, c = styles.get(pname, ("-", "o", "gray"))
        ax.plot(valid_ratios, costs, ls, marker=marker, color=c,
                label=pname, linewidth=2, markersize=7)

ax.set_xlabel("Cost Ratio (C_event / C_visit)")
ax.set_ylabel("Mean Expected Cost per Patient (EUR)")
ax.set_title("Cost Sensitivity: How Cost Ratio Affects Policy Performance")
ax.legend()
plt.tight_layout()
plt.show()

### 3.3 Uncertainty Alpha Sensitivity (H2)

**Hypothesis H2**: Incorporating prediction uncertainty into scheduling improves expected utility.

In [ ]:
from evaluation.sensitivity import vary_uncertainty_alpha

alphas = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
alpha_results = vary_uncertainty_alpha(
    survival_curves=survival_curves,
    survival_stds=survival_stds,
    event_times=event_times,
    event_indicators=event_indicators,
    alphas=alphas,
    is_heart_failure=is_hf,
)

# Extract ILP-Uncertainty results across alpha
unc_ebf = []
unc_cost = []
valid_alphas = []
for a in alphas:
    if "CCPFS-ILP-Uncertainty" in alpha_results[a]:
        unc_ebf.append(alpha_results[a]["CCPFS-ILP-Uncertainty"]["ebf"]["ebf_rate"])
        unc_cost.append(alpha_results[a]["CCPFS-ILP-Uncertainty"]["cost"]["mean_cost"])
        valid_alphas.append(a)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(valid_alphas, unc_ebf, "-D", color="#2980b9", linewidth=2, markersize=7)
ax1.set_xlabel("Uncertainty Alpha")
ax1.set_ylabel("EBF Rate")
ax1.set_title("H2: EBF Rate vs Uncertainty Conservatism")

ax2.plot(valid_alphas, unc_cost, "-D", color="#e74c3c", linewidth=2, markersize=7)
ax2.set_xlabel("Uncertainty Alpha")
ax2.set_ylabel("Mean Expected Cost (EUR)")
ax2.set_title("H2: Expected Cost vs Uncertainty Conservatism")

plt.tight_layout()
plt.show()

## 4. Example Patient Recommendations

Pick one patient from each risk group and show the full pipeline output.

In [ ]:
from policy.cost_function import expected_cost_curve

# Pick representative patients
ilp_assignments = None
for pname, r in results.items():
    if pname == "CCPFS-ILP":
        # Re-run to get assignments
        from policy.ilp_scheduler import schedule_ilp
        ilp_result = schedule_ilp(survival_curves, capacity)
        ilp_assignments = ilp_result["assignments"]
        break

examples = {}
for g, label in [(2, "High Risk"), (1, "Medium Risk"), (0, "Low Risk")]:
    candidates = np.where(risk_groups == g)[0]
    # Pick the patient closest to group median 30-day risk
    risks = 1.0 - survival_curves[candidates, 30]
    median_risk = np.median(risks)
    best = candidates[np.argmin(np.abs(risks - median_risk))]
    examples[label] = int(best)

print("Selected example patients:")
for label, idx in examples.items():
    r30 = 1.0 - survival_curves[idx, 30]
    assigned = ilp_assignments[idx] if ilp_assignments else "N/A"
    event = f"day {event_times[idx]}" if event_indicators[idx] else "none"
    print(f"  {label} (patient {idx}): R(30)={r30:.2%}, assigned=day {assigned}, event={event}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
days = np.arange(0, 31)
group_colours = {"High Risk": "#e74c3c", "Medium Risk": "#f39c12", "Low Risk": "#2ecc71"}

for ax, (label, idx) in zip(axes, examples.items()):
    S = survival_curves[idx]
    S_std = survival_stds[idx]
    R = 1.0 - S
    c = group_colours[label]

    # Risk curve with CI
    ax.plot(days, R, color=c, linewidth=2, label="Predicted risk")
    ax.fill_between(days,
                    np.clip(R - 1.96 * S_std, 0, 1),
                    np.clip(R + 1.96 * S_std, 0, 1),
                    color=c, alpha=0.15, label="95% CI")

    # Assigned follow-up day
    if ilp_assignments and idx in ilp_assignments:
        d = ilp_assignments[idx]
        ax.axvline(x=d, color="#2980b9", linestyle="-", linewidth=2,
                   label=f"CCPFS: day {d}")

    # Uniform-14 for comparison
    ax.axvline(x=14, color="gray", linestyle="--", alpha=0.5, label="Uniform: day 14")

    # Actual event (if any)
    if event_indicators[idx]:
        ax.axvline(x=event_times[idx], color="black", linestyle=":",
                   linewidth=1.5, label=f"Event: day {event_times[idx]}")

    ax.set_xlabel("Days since discharge")
    ax.set_ylabel("Cumulative risk R(t)")
    ax.set_title(f"{label} (Patient {idx})")
    ax.legend(fontsize=8, loc="upper left")
    ax.set_xlim(0, 30)
    ax.set_ylim(0, None)

fig.suptitle("Patient Risk Curves with Follow-Up Recommendations", fontsize=13)
plt.tight_layout()
plt.show()

## 5. Summary

### Key Findings (Synthetic Data)

These results use synthetic Weibull survival curves. When real MIMIC-IV survival models are trained, the same notebook structure will produce paper-ready figures.

**What to look for:**
- CCPFS-ILP should have lower EBF rate than all baselines under the same capacity
- The advantage should grow as capacity shrinks (H3 plot)
- Uncertainty adjustment should reduce EBF rate at the cost of slightly higher expected cost (H2 plot)
- High-risk patients get earlier follow-up days; low-risk patients are deferred

In [ ]:
print("\nFinal comparison table:\n")
print(results_summary_table(results))